# Processing invoices using Box's MCP Server

In [15]:
import os
import json
from dotenv import load_dotenv
from google import genai
from google.genai import types
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from database import setup_database, update_invoice_in_database, generate_report
from model import generate

load_dotenv(override=True)

True

Let's now configure a few settings that we'll use throughout the notebook.

In [16]:
BOX_CLIENT_ID = os.getenv("BOX_CLIENT_ID")
BOX_CLIENT_SECRET = os.getenv("BOX_CLIENT_SECRET")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
BOX_FOLDER_ID = os.getenv("BOX_FOLDER_ID")
BOX_MCP_SERVER_PATH = "/Users/svpino/dev/mcp-server-box"
BOX_MCP_TOOLS = [
    "box_who_am_i",
    "box_list_folder_content_by_folder_id",
    "box_ai_extract_tool",
]

## Configuring Box' MCP Server

First, let's configure Gemini and prepare the connection to the Box's MCP Server.

In [17]:
client = genai.Client(api_key=GEMINI_API_KEY)

server_params = StdioServerParameters(
    command="uv",
    args=[
        "--directory",
        BOX_MCP_SERVER_PATH,
        "run",
        "src/mcp_server_box.py",
    ],
)

Now, let's create a function to set up the list of MCP tools that will be used to interact with Box.

In [18]:
async def get_mcp_tools(session: ClientSession):
    """
    Set up the list of MCP tools that will be used to interact with Box.
    """
    mcp_tools = await session.list_tools()
    return [
        types.Tool(
            function_declarations=[
                {
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": {p: v for p, v in tool.inputSchema.items()},
                }
            ]
        )
        for tool in mcp_tools.tools
        if tool.name in BOX_MCP_TOOLS
    ]

## Processing invoices

Let's start by defining the function that will extract the data from a given invoice.

In [19]:
async def extract_invoice_fields(
    invoice: dict, client: genai.Client, session: ClientSession, tools: list
):
    """
    Extract data from the given invoice.
    """
    print(f'Extracting data from invoice "{invoice["name"]}"...')
    prompt = (
        "Extract the following fields from the invoice "
        f"with file_id {invoice['id']}: "
        "client_name, invoice_amount, product_name. "
        "Return the invoice_amount as a float. "
    )
    response = await generate(prompt, client, session, tools)
    result = json.loads(response[0]["answer"])
    result["file"] = invoice["name"]
    return result


Let's set up the database where we'll store the information of every invoice.

In [20]:
connection = setup_database()

Let's create a Gemini session with access to Box's MCP Server, list the invoices stored in Box and extract the data from each of them.

In [21]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        tools = await get_mcp_tools(session)

        # Get the list of invoices from the Box folder.
        invoices = await generate(
            f"List the content of the folder with id {BOX_FOLDER_ID}",
            client,
            session,
            tools,
        )
        print(f"Found {len(invoices)} invoices")

        # Process each invoice and extract the required fields.
        for invoice in invoices:
            invoice_data = await extract_invoice_fields(invoice, client, session, tools)
            print(f"Extracted data: {invoice_data}")
            update_invoice_in_database(connection, invoice_data)

[Calling MCP Tool: "box_list_folder_content_by_folder_id"]
Found 4 invoices
Extracting data from invoice "Invoice-ETOX6AZS-0004.pdf"...
[Calling MCP Tool: "box_ai_extract_tool"]
Extracted data: {'client_name': 'Santiago Valdarrama', 'invoice_amount': '15.00', 'product_name': 'myproduct', 'file': 'Invoice-ETOX6AZS-0004.pdf'}
Updating database with invoice Invoice-ETOX6AZS-0004.pdf...
Extracting data from invoice "Invoice-JV3QQ0DZ-0005.pdf"...
[Calling MCP Tool: "box_ai_extract_tool"]
Extracted data: {'client_name': 'Acme Inc', 'invoice_amount': 400.0, 'product_name': 'Machine Learning School', 'file': 'Invoice-JV3QQ0DZ-0005.pdf'}
Updating database with invoice Invoice-JV3QQ0DZ-0005.pdf...
Extracting data from invoice "Invoice-JV3QQ0DZ-0006.pdf"...
[Calling MCP Tool: "box_ai_extract_tool"]
Extracted data: {'client_name': 'Acme Inc', 'invoice_amount': 400.0, 'product_name': 'Machine Learning School', 'file': 'Invoice-JV3QQ0DZ-0006.pdf'}
Updating database with invoice Invoice-JV3QQ0DZ-0006

## Generating final report

Finally, we want to generate a report with the data that we stored in the database.

In [14]:
generate_report(connection)
connection.close()


Invoice Report
* Total invoices: 4
* Total amount: 908.5

Breakdown by client:
* Acme Inc: 2 invoices ($800.0)
* Santiago Valdarrama: 1 invoices ($15.0)
* Test Business: 1 invoices ($93.5)
